# Lab1 summarize dialogue

In [1]:
#Install Libraries
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
#load dataset
from datasets import load_dataset

dataset = load_dataset("npc-engine/light-batch-summarize-dialogue")

  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['dialogue_text', 't0pp_prediction'],
        num_rows: 17075
    })
    test: Dataset({
        features: ['dialogue_text', 't0pp_prediction'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['dialogue_text', 't0pp_prediction'],
        num_rows: 1000
    })
})

In [6]:
dataset["train"][0]['dialogue_text']

'court wizard: Have any else come up this eve? I had hoped for a quiet night to examine the stars\nsoldier: Yes, a few came through, but it is a cold night for me, I am used to warmer weather\ncourt wizard: Well, you are but a common soldier.  No doubt you are used to such a lot.  Thankfully I have my spells to keep me warm.\nsoldier: I am a soldier doing my job\ncourt wizard: Yes... well... Very well then.  See that you do!  No slacking off while your betters are about.\nsoldier: No sir\ncourt wizard: When, for example, was this horn last tested?  It looks dented.  How can we be sure it will work?\nsoldier: A year ago, Test it out or cause a need to use it\ncourt wizard: Mayhap I will speak to the king about such lackness.  Or perhaps I can sell him a spell that will serve just as well.\nsoldier: Good idea, I agree, go do that\ncourt wizard: Get off of me, you fool!  Who gave you permission to touch me!\nsoldier: To the jail with you\n\nSummarize the dialogue'

In [7]:
dataset['train'][0]['t0pp_prediction']

'court wizard is cold on the night he is examining the stars. Soldier is a soldier doing his job.'

In [2]:
#Loop over the Dataset
example_indices = [0, 10]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue_text'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['t0pp_prediction'])
    print(dash_line)

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:


NameError: name 'dataset' is not defined

## Summarize Dialogue without Prompt Engineering

In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

Load the FLAN-T5 model, creating an instance of the AutoModelForSeq2SeqLM class with the .from_pretrained() method.

In [6]:
#Use Model flan-t5
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

To perform encoding and decoding, we work with text in a tokenized form. Tokenization is the process of splitting texts into smaller units that can be processed by the LLM models.

In [7]:
#Tokenise the dataset
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Now it's time to explore how well the base LLM summarizes a dialogue without any prompt engineering. Prompt engineering is an act of a human changing the prompt (input) to improve the response for a given task.

In [8]:
#Selecting a index to be given as a prompt to the model
for i, index in enumerate(example_indices):
    dialogue = dataset['train'][index]['dialogue_text']
    summary = dataset['train'][index]['t0pp_prediction']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
court wizard: Have any else come up this eve? I had hoped for a quiet night to examine the stars
soldier: Yes, a few came through, but it is a cold night for me, I am used to warmer weather
court wizard: Well, you are but a common soldier.  No doubt you are used to such a lot.  Thankfully I have my spells to keep me warm.
soldier: I am a soldier doing my job
court wizard: Yes... well... Very well then.  See that you do!  No slacking off while your betters are about.
soldier: No sir
court wizard: When, for example, was this horn last tested?  It looks dented.  How can we be sure it will work?
soldier: A year ago, Test it out or cause a need to use it
court wizard: Mayhap I will speak to the king about such lackness.  Or perhaps I can sell him a spell that will serve

## Summarize Dialogue with an Instruction Prompt
Prompt engineering is an important concept in using foundation models for text generation. You can check out this blog from Amazon Science for a quick introduction to prompt engineering.


### Zero Shot Inference with an Instruction Prompt
In order to instruct the model to perform a task - summarize a dialogue - you can take the dialogue and convert it into an instruction prompt. This is often called zero shot inference. You can check out this blog from AWS for a quick description of what zero shot learning is and why it is an important concept to the LLM model.

Wrap the dialogue in a descriptive instruction and see how the generated text will change:

In [9]:
for i, index in enumerate(example_indices):
    dialogue = dataset['train'][index]['dialogue_text']
    summary = dataset['train'][index]['t0pp_prediction']

    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

court wizard: Have any else come up this eve? I had hoped for a quiet night to examine the stars
soldier: Yes, a few came through, but it is a cold night for me, I am used to warmer weather
court wizard: Well, you are but a common soldier.  No doubt you are used to such a lot.  Thankfully I have my spells to keep me warm.
soldier: I am a soldier doing my job
court wizard: Yes... well... Very well then.  See that you do!  No slacking off while your betters are about.
soldier: No sir
court wizard: When, for example, was this horn last tested?  It looks dented.  How can we be sure it will work?
soldier: A year ago, Test it out or cause a need to use it
court wizard: Mayhap I will speak to the king about such lackness.  Or perhap

### 3.2 - Zero Shot Inference with the Prompt Template from FLAN-T5
Let's use a slightly different prompt. FLAN-T5 has many prompt templates that are published for certain tasks here. In the following code, we use one of the pre-built FLAN-T5 prompts:

In [10]:
for i, index in enumerate(example_indices):
    dialogue = dataset['train'][index]['dialogue_text']
    summary = dataset['train'][index]['t0pp_prediction']

    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

court wizard: Have any else come up this eve? I had hoped for a quiet night to examine the stars
soldier: Yes, a few came through, but it is a cold night for me, I am used to warmer weather
court wizard: Well, you are but a common soldier.  No doubt you are used to such a lot.  Thankfully I have my spells to keep me warm.
soldier: I am a soldier doing my job
court wizard: Yes... well... Very well then.  See that you do!  No slacking off while your betters are about.
soldier: No sir
court wizard: When, for example, was this horn last tested?  It looks dented.  How can we be sure it will work?
soldier: A year ago, Test it out or cause a need to use it
court wizard: Mayhap I will speak to the king about such lackness.  Or perhaps I can sell him a spell tha

## Summarize Dialogue with One Shot and Few Shot Inference
One shot and few shot inference are the practices of providing an LLM with either one or more full examples of prompt-response pairs that match your task - before your actual prompt that you want completed. This is called "in-context learning" and puts your model into a state that understands your specific task. You can read more about it in this blog from HuggingFace.


### One Shot Inference
Let's build a function that takes a list of example_indices_full, generates a prompt with full examples, then at the end appends the prompt which you want the model to complete (example_index_to_summarize).

In [11]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['train'][index]['dialogue_text']
        summary = dataset['train'][index]['t0pp_prediction']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""

    dialogue = dataset['train'][example_index_to_summarize]['dialogue_text']

    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""

    return prompt

In [13]:
example_indices_full = [0]
example_index_to_summarize = 100

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

court wizard: Have any else come up this eve? I had hoped for a quiet night to examine the stars
soldier: Yes, a few came through, but it is a cold night for me, I am used to warmer weather
court wizard: Well, you are but a common soldier.  No doubt you are used to such a lot.  Thankfully I have my spells to keep me warm.
soldier: I am a soldier doing my job
court wizard: Yes... well... Very well then.  See that you do!  No slacking off while your betters are about.
soldier: No sir
court wizard: When, for example, was this horn last tested?  It looks dented.  How can we be sure it will work?
soldier: A year ago, Test it out or cause a need to use it
court wizard: Mayhap I will speak to the king about such lackness.  Or perhaps I can sell him a spell that will serve just as well.
soldier: Good idea, I agree, go do that
court wizard: Get off of me, you fool!  Who gave you permission to touch me!
soldier: To the jail with you

Summarize the dialogue

What was going on?
court w

Now pass this prompt to perform the one shot inference:

In [14]:
summary = dataset['train'][example_index_to_summarize]['t0pp_prediction']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
the groundskeeper of the castle will buy some curtains and a mattress for the goldfish's room.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
The groundskeeper of the castle will go into town tomorrow to find some curtains and a cotton mattress. The groundskeeper of the castle will drop pet goldfish off by the pond to get some fresh air.


### Few Shot Inference

Let's explore few shot inference by adding two more full dialogue-summary pairs to your prompt.

In [15]:
example_indices_full = [0, 10, 100]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

court wizard: Have any else come up this eve? I had hoped for a quiet night to examine the stars
soldier: Yes, a few came through, but it is a cold night for me, I am used to warmer weather
court wizard: Well, you are but a common soldier.  No doubt you are used to such a lot.  Thankfully I have my spells to keep me warm.
soldier: I am a soldier doing my job
court wizard: Yes... well... Very well then.  See that you do!  No slacking off while your betters are about.
soldier: No sir
court wizard: When, for example, was this horn last tested?  It looks dented.  How can we be sure it will work?
soldier: A year ago, Test it out or cause a need to use it
court wizard: Mayhap I will speak to the king about such lackness.  Or perhaps I can sell him a spell that will serve just as well.
soldier: Good idea, I agree, go do that
court wizard: Get off of me, you fool!  Who gave you permission to touch me!
soldier: To the jail with you

Summarize the dialogue

What was going on?
court w

Now pass this prompt to perform a few shot inference:

In [16]:
summary = dataset['train'][example_index_to_summarize]['t0pp_prediction']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
boar gives peasant flowers for his wife. He offers peasant seeds that will make him a noble.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
Peasant and boar are going to get flowers for their wife.


## Generative Configuration Parameters for Inference
We can change the configuration parameters of the generate() method to see a different output from the LLM. So far the only parameter that you have been setting was max_new_tokens=50, which defines the maximum number of tokens to generate. A full list of available parameters can be found in the Hugging Face Generation documentation.

A convenient way of organizing the configuration parameters is to use GenerationConfig class.

Exercise:

Change the configuration parameters to investigate their influence on the output.

Putting the parameter do_sample = True, you activate various decoding strategies which influence the next token from the probability distribution over the entire vocabulary. You can then adjust the outputs changing temperature and other parameters (such as top_k and top_p).

In [17]:
generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
Peasant and boar are going to get flowers for their wife.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
boar gives peasant flowers for his wife. He offers peasant seeds that will make him a noble.



In [18]:
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
Peasant and boar are helping to find a gift for their wife. Boar will plant seeds for her.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
boar gives peasant flowers for his wife. He offers peasant seeds that will make him a noble.



Comments related to the choice of the parameters in the code cell above:

* Choosing max_new_tokens less will make the output text too short, so the dialogue summary will be cut.
* Putting do_sample = True and changing the temperature value you get more flexibility in the output.

# Lab 2 Fine-Tune a Generative AI Model for Dialogue Summarization

In [4]:
%pip install \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 14.4 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [3]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Test the Model with Zero Shot Inferencing


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['dialogue_text', 't0pp_prediction'],
        num_rows: 17075
    })
    test: Dataset({
        features: ['dialogue_text', 't0pp_prediction'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['dialogue_text', 't0pp_prediction'],
        num_rows: 1000
    })
})

In [9]:
dialogue = dataset['train'][0]['dialogue_text']
summary = dataset['train'][0]['t0pp_prediction']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

court wizard: Have any else come up this eve? I had hoped for a quiet night to examine the stars
soldier: Yes, a few came through, but it is a cold night for me, I am used to warmer weather
court wizard: Well, you are but a common soldier.  No doubt you are used to such a lot.  Thankfully I have my spells to keep me warm.
soldier: I am a soldier doing my job
court wizard: Yes... well... Very well then.  See that you do!  No slacking off while your betters are about.
soldier: No sir
court wizard: When, for example, was this horn last tested?  It looks dented.  How can we be sure it will work?
soldier: A year ago, Test it out or cause a need to use it
court wizard: Mayhap I will speak to the king about such lackness.  Or perhaps I can sell him a spell that will serve just as well.
soldier: Good idea, I agree, go do that
court wizard: Ge

# Perform Full Fine-Tuning

## 1. Preprocess the Dialog-Summary Dataset

In [4]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue_text"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["t0pp_prediction"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['dialogue_text', 't0pp_prediction'])

In [5]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

In [6]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (171, 2)
Validation: (10, 2)
Test: (20, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 171
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 20
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 10
    })
})


### 2. Fine-Tune the Model with the Preprocessed Dataset


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
output_dir = f'/content/drive/MyDrive/Advanced_DS/WorkedExample'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_steps=100,
    save_steps=100,
    warmup_steps=10,
    prediction_loss_only=True
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [11]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=215, training_loss=5.155523255813954, metrics={'train_runtime': 340.0497, 'train_samples_per_second': 2.514, 'train_steps_per_second': 0.632, 'total_flos': 585467536343040.0, 'train_loss': 5.155523255813954, 'epoch': 5.0})

### Evaluate the model

In [12]:
eval_results = trainer.evaluate()
eval_results['eval_loss']

4.4375

In [13]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Advanced_DS/WorkedExample/checkpoint-200", torch_dtype=torch.bfloat16)


In [16]:
dialogue = dataset['train'][0]['dialogue_text']
human_baseline_summary = dataset['train'][0]['t0pp_prediction']
dash_line = '-'.join('' for x in range(100))

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids


instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
court wizard is cold on the night he is examining the stars. Soldier is a soldier doing his job.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
Soldier is a common soldier. He is used to warmer weather.


### Evaluate the Model Quantitatively (with ROUGE Metric)


In [17]:
rouge = evaluate.load('rouge')

In [23]:
dialogues = dataset['train'][0:10]['dialogue_text']
human_baseline_summaries = dataset['train'][0:10]['t0pp_prediction']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to('cuda:0')

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    input_ids = input_ids.to(instruct_model.device)
    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,court wizard is cold on the night he is examin...,Soldier is a common soldier. He is used to war...,Soldier is a common soldier. He is used to war...
1,soldier is a soldier doing his job. Court wiza...,Soldier is a common soldier. He is used to war...,Soldier is a common soldier. He is used to war...
2,"thief wants to buy a beer for a homeless man, ...",The thief is going to buy a beer for the person.,The thief is going to buy a beer for the person.
3,thief wants to buy a beer for a homeless man b...,The thief is looking down on the person.,The thief is looking down on the person.
4,witch and fairy are stuck in the tower. They a...,"Fairy is having a bad day. She has Felix, but ...","Fairy is having a bad day. She has Felix, but ..."
5,witch is locked in a room with a fairy. She ca...,"Fairy is locked in. She has Felix, her familia...","Fairy is locked in. She has Felix, her familia..."
6,The king wants to see the prisoner. The prison...,The king wants to see prisoner.,The king wants to see prisoner.
7,The king wants to see the prisoner. The prison...,The king wants to see prisoner.,The king wants to see prisoner.
8,the king's bed frame is made of gold. archer w...,The king's bed frame is more valuable than his...,The king's bed frame is more valuable than his...
9,the king has invited archer to his castle to t...,Archer is a good archer. The king is a good heir.,Archer is a good archer. The king is a good heir.


Evaluate the models computing ROUGE metrics. Notice the improvement in the results!

In [24]:
# Evaluate the models computing ROUGE metrics. 
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.3516232535782976, 'rouge2': 0.20553465562259934, 'rougeL': 0.3092992752080421, 'rougeLsum': 0.3097385589336634}
INSTRUCT MODEL:
{'rouge1': 0.3583838430922206, 'rouge2': 0.20426774483378254, 'rougeL': 0.310671297596006, 'rougeLsum': 0.3110050218427736}


In [25]:
print("Absolute percentage improvement of INSTRUCT MODEL over HUMAN BASELINE")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of INSTRUCT MODEL over HUMAN BASELINE
rouge1: 0.68%
rouge2: -0.13%
rougeL: 0.14%
rougeLsum: 0.13%


### Setup the PEFT/LoRA model for Fine-Tuning


In [38]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [39]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))


trainable model parameters: 3538944
all model parameters: 251116800



In [40]:
output_dir = f'/content/drive/MyDrive/Advanced_DS/peft-dialogue-summary-training'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_steps=200,
    save_steps=200,
    warmup_steps=10,
    prediction_loss_only=True
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

In [41]:
peft_trainer.train()

peft_model_path="/content/drive/MyDrive/Advanced_DS/peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


('/content/drive/MyDrive/Advanced_DS/peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 '/content/drive/MyDrive/Advanced_DS/peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 '/content/drive/MyDrive/Advanced_DS/peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [42]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       '/content/drive/MyDrive/Advanced_DS/peft-dialogue-summary-checkpoint-local/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

###  Evaluate the Model Qualitatively (Human Evaluation)

In [45]:
dialogue = dataset['train'][0]['dialogue_text']
baseline_human_summary = dataset['train'][0]['t0pp_prediction']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to('cuda:0')

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

input_ids = input_ids.to(instruct_model.device)
instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

input_ids = input_ids.to(peft_model.device)
peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
court wizard is cold on the night he is examining the stars. Soldier is a soldier doing his job.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
Soldier is a soldier.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
Soldier is a common soldier. He is used to warmer weather.
---------------------------------------------------------------------------------------------------
PEFT MODEL: Soldier is a common soldier. He is used to warmer weather. He has spells to keep him warm.


###  Evaluate the Model Quantitatively (with ROUGE Metric)

Perform inferences for the sample of the test dataset (only 10 dialogues and summaries to save time).

In [46]:
dialogues = dataset['train'][0:10]['dialogue_text']
human_baseline_summaries = dataset['train'][0:10]['t0pp_prediction']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to('cuda:0')

    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    input_ids = input_ids.to(instruct_model.device)
    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    input_ids = input_ids.to(peft_model.device)
    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])

['court wizard is cold on the night he is examining the stars. Soldier is a soldier doing his job.',
 'soldier is a soldier doing his job. Court wizard is a court wizard. He wants to know when the horn was last tested. It was last tested a year ago.',
 "thief wants to buy a beer for a homeless man, but he doesn't have enough money.",
 'thief wants to buy a beer for a homeless man but has no money.',
 'witch and fairy are stuck in the tower. They are thinking of ways to get out.',
 "witch is locked in a room with a fairy. She can't get out. She can't leave her family behind. She can't let the lantern down. She can't use magic. She has a cat as a familiar. The cat can't fit through the bars",
 "The king wants to see the prisoner. The prisoner is accused of disobeying the queen's order.",
 "The king wants to see the prisoner. The prisoner is accused of disobeying the queen's order.",
 'the king has invited archer to his castle to teach him archery.',
 'princess wants to cross the bridge t

In [47]:
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries,peft_model_summaries
0,court wizard is cold on the night he is examin...,Soldier is cold.,Soldier is a common soldier. He is used to war...,Soldier is a common soldier. He is used to war...
1,soldier is a soldier doing his job. Court wiza...,The soldier's horn is a bit dented.,Soldier is a common soldier. He is used to war...,Soldier is a common soldier. He is used to war...
2,"thief wants to buy a beer for a homeless man, ...",The thief is a homeless man.,The thief is going to buy a beer for the person.,The thief is looking for a bite to eat.
3,thief wants to buy a beer for a homeless man b...,The thief is a homeless man.,The thief is looking down on the person.,The thief is looking for a beer. The person is...
4,witch and fairy are stuck in the tower. They a...,The fairy is having a bad state of affairs. Sh...,"Fairy is having a bad day. She has Felix, but ...","Fairy is having a bad day. She has Felix, her ..."
5,witch is locked in a room with a fairy. She ca...,Fairy is locked in a room.,"Fairy is locked in. She has Felix, her familia...","Fairy is locked in the house. She has Felix, h..."
6,The king wants to see the prisoner. The prison...,The king wants to make a deal with the prisoner.,The king wants to see prisoner.,The king wants to see prisoner. The king wants...
7,The king wants to see the prisoner. The prison...,The king is requesting to meet with prisoner.,The king wants to see prisoner.,The king wants to see prisoner. The king wants...
8,the king has invited archer to his castle to t...,The king is a proud to have his bed frame and ...,Archer is a good archer. The king is a good heir.,Archer is a good archer. The king is a good ar...
9,princess wants to cross the bridge to leave he...,Princess needs to cross the bridge. She has a ...,Princess needs to cross the bridge to leave he...,Princess needs to cross the bridge to leave he...


In [48]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.33320098005223864, 'rouge2': 0.1600459057850362, 'rougeL': 0.2882529027883719, 'rougeLsum': 0.292340444875914}
INSTRUCT MODEL:
{'rouge1': 0.3863983003419402, 'rouge2': 0.22697422855721544, 'rougeL': 0.333065884274792, 'rougeLsum': 0.33917084676937254}
PEFT MODEL:
{'rouge1': 0.3869715955558476, 'rouge2': 0.2083942168942169, 'rougeL': 0.33320506964696345, 'rougeLsum': 0.3410040274459213}


Calculate the improvement of PEFT over the original model:

In [49]:
print("Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE
rouge1: 5.38%
rouge2: 4.83%
rougeL: 4.50%
rougeLsum: 4.87%


Now calculate the improvement of PEFT over a full fine-tuned model:

In [50]:
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL
rouge1: 0.06%
rouge2: -1.86%
rougeL: 0.01%
rougeLsum: 0.18%


# Lab 3 Fine-Tune FLAN-T5 with Reinforcement Learning (PPO) and PEFT to Generate Less-Toxic Summaries

In [16]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

%pip install trl==0.7.2

  Using cached trl-0.7.2-py3-none-any.whl.metadata (10 kB)
Using cached trl-0.7.2-py3-none-any.whl (124 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.7.4
    Uninstalling trl-0.7.4:
      Successfully uninstalled trl-0.7.4


In [19]:
%pip install git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-b6kiugs5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-b6kiugs5
  Resolved https://github.com/huggingface/peft.git to commit 0ae52fece17a3514116815984444116b75d9c5ca
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

import torch
import evaluate

import numpy as np
import pandas as pd

# tqdm library makes the loops show a smart progress meter.
from tqdm import tqdm
tqdm.pandas()

In [3]:
# trl: Transformer Reinforcement Learning library
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

In [4]:
model_name="google/flan-t5-base"
huggingface_dataset_name = "npc-engine/light-batch-summarize-dialogue"

dataset_original = load_dataset(huggingface_dataset_name)

dataset_original

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['dialogue_text', 't0pp_prediction'],
        num_rows: 17075
    })
    test: Dataset({
        features: ['dialogue_text', 't0pp_prediction'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['dialogue_text', 't0pp_prediction'],
        num_rows: 1000
    })
})

In [5]:
def build_dataset(model_name,
                  dataset_name,
                  input_min_text_length,
                  input_max_text_length):

    """
    Preprocess the dataset and split it into train and test parts.

    Parameters:
    - model_name (str): Tokenizer model name.
    - dataset_name (str): Name of the dataset to load.
    - input_min_text_length (int): Minimum length of the dialogues.
    - input_max_text_length (int): Maximum length of the dialogues.

    Returns:
    - dataset_splits (datasets.dataset_dict.DatasetDict): Preprocessed dataset containing train and test parts.
    """

    # load dataset (only "train" part will be enough for this lab).
    dataset = load_dataset(dataset_name, split="train")

    # Filter the dialogues of length between input_min_text_length and input_max_text_length characters.
    dataset = dataset.filter(lambda x: len(x["dialogue_text"]) > input_min_text_length and len(x["dialogue_text"]) <= input_max_text_length, batched=False)

    # Prepare tokenizer. Setting device_map="auto" allows to switch between GPU and CPU automatically.
    tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

    def tokenize(sample):

        # Wrap each dialogue with the instruction.
        prompt = f"""
Summarize the following conversation.

{sample["dialogue_text"]}

Summary:
"""
        sample["input_ids"] = tokenizer.encode(prompt)

        # This must be called "query", which is a requirement of our PPO library.
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    # Tokenize each dialogue.
    dataset = dataset.map(tokenize, batched=False)
    dataset.set_format(type="torch")

    # Split the dataset into train and test parts.
    dataset_splits = dataset.train_test_split(test_size=0.2, shuffle=False, seed=42)

    return dataset_splits

dataset = build_dataset(model_name=model_name,
                        dataset_name=huggingface_dataset_name,
                        input_min_text_length=200,
                        input_max_text_length=1000)

print(dataset)

Map:   0%|          | 0/15643 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['dialogue_text', 't0pp_prediction', 'input_ids', 'query'],
        num_rows: 12514
    })
    test: Dataset({
        features: ['dialogue_text', 't0pp_prediction', 'input_ids', 'query'],
        num_rows: 3129
    })
})


Prepare a function to pull out the number of model parameters (it is the same as in the previous lab):

In [6]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\n"

Add the adapter to the original FLAN-T5 model. In the previous lab you were adding the fully trained adapter only for inferences, so there was no need to pass LoRA configurations doing that. Now you need to pass them to the constructed PEFT model, also putting is_trainable=True.

In [7]:
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
                                              torch_dtype=torch.bfloat16)

peft_model = PeftModel.from_pretrained(model,
                                       '/content/drive/MyDrive/Advanced_DS/peft-dialogue-summary-checkpoint-local/',
                                       lora_config=lora_config,
                                       torch_dtype=torch.bfloat16,
                                       device_map="auto",
                                       is_trainable=True)

print(f'PEFT model parameters to be updated:\n{print_number_of_trainable_model_parameters(peft_model)}\n')


PEFT model parameters to be updated:

trainable model parameters: 3538944
all model parameters: 251116800




In this lab, we are preparing to fine-tune the LLM using Reinforcement Learning (RL). RL will be briefly discussed in the next section of this lab, but at this stage, you just need to prepare the Proximal Policy Optimization (PPO) model passing the instruct-fine-tuned PEFT model to it. PPO will be used to optimize the RL policy against the reward model.

In [8]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model,
                                                               torch_dtype=torch.bfloat16,
                                                               is_trainable=True)

print(f'PPO model parameters to be updated (ValueHead + 769 params):\n{print_number_of_trainable_model_parameters(ppo_model)}\n')
print(ppo_model.v_head)

PPO model parameters to be updated (ValueHead + 769 params):

trainable model parameters: 3539713
all model parameters: 251117569


ValueHead(
  (dropout): Dropout(p=0.1, inplace=False)
  (summary): Linear(in_features=768, out_features=1, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


During PPO, only a few parameters will be updated. Specifically, the parameters of the ValueHead. More information about this class of models can be found in the documentation. The number of trainable parameters can be computed as
, where
 is the number of input units (here
) and
 is the number of output units (you have
). The
 term in the equation takes into account the bias term.

Now create a frozen copy of the PPO which will not be fine-tuned - a reference model. The reference model will represent the LLM before detoxification. None of the parameters of the reference model will be updated during PPO training. This is on purpose.

In [9]:
ref_model = create_reference_model(ppo_model)

print(f'Reference model parameters to be updated:\n{print_number_of_trainable_model_parameters(ref_model)}\n')

Reference model parameters to be updated:

trainable model parameters: 0
all model parameters: 251117569




## Prepare Reward Model
Reinforcement Learning (RL) is one type of machine learning where agents take actions in an environment aimed at maximizing their cumulative rewards. The agent's behavior is defined by the policy. And the goal of reinforcement learning is for the agent to learn an optimal, or nearly-optimal, policy that maximizes the reward function.

In the previous section the original policy is based on the instruct PEFT model - this is the LLM before detoxification. Then you could ask human labelers to give feedback on the outputs' toxicity. However, it can be expensive to use them for the entire fine-tuning process. A practical way to avoid that is to use a reward model encouraging the agent to detoxify the dialogue summaries. The intuitive approach would be to do some form of sentiment analysis across two classes (nothate and hate) and give a higher reward if there is higher a chance of getting class nothate as an output.

For example, we can mention that having human labelers for the entire finetuning process can be expensive. A practical way to avoid that is to use a reward model.

use feedback generated by a model

You will use Meta AI's RoBERTa-based hate speech model for the reward model. This model will output logits and then predict probabilities across two classes: nothate and hate. The logits of the output nothate will be taken as a positive reward. Then, the model will be fine-tuned with PPO using those reward values.

Create the instance of the required model class for the RoBERTa model. You also need to load a tokenizer to test the model. Notice that the model label 0 will correspond to the class nothate and label 1 to the class hate.

In [10]:
toxicity_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = AutoTokenizer.from_pretrained(toxicity_model_name, device_map="auto")
toxicity_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name, device_map="auto")
print(toxicity_model.config.id2label)

{0: 'nothate', 1: 'hate'}


Take some non-toxic text, tokenize it, and pass it to the model. Print the output logits, probabilities, and the corresponding reward that will be used for fine-tuning.

In [11]:
non_toxic_text = "#Person 1# tells Tommy that he didn't like the movie."

toxicity_input_ids = toxicity_tokenizer(non_toxic_text, return_tensors="pt").input_ids

toxicity_input_ids = toxicity_input_ids.to('cuda:0')
logits = toxicity_model(input_ids=toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# Print the probabilities for [not hate, hate]
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# get the logits for "not hate" - this is the reward!
not_hate_index = 0
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (high): {nothate_reward}')

logits [not hate, hate]: [3.114100694656372, -2.4896175861358643]
probabilities [not hate, hate]: [0.9963293671607971, 0.0036706167738884687]
reward (high): [3.114100694656372]


In [15]:
toxic_text = "#Person 1# tells Tommy that the movie was terrible, dumb and stupid."

toxicity_input_ids = toxicity_tokenizer(toxic_text, return_tensors="pt").input_ids

toxicity_input_ids = toxicity_input_ids.to('cuda:0')
logits = toxicity_model(toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# Print the probabilities for [not hate, hate]
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# Get the logits for "not hate" - this is the reward!
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (low): {nothate_reward}')

logits [not hate, hate]: [-0.6921164393424988, 0.37227070331573486]
probabilities [not hate, hate]: [0.2564719617366791, 0.7435280084609985]
reward (low): [-0.6921164393424988]


Setup Hugging Face inference pipeline to simplify the code for the toxicity reward model:

In [16]:
%pip install transformers==4.34.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.3
    Uninstalling huggingface-hub-0.19.3:
      Successfully uninstalled huggingface-hub-0.19.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.27.2
    Uninstalling transformers-4.27.2:
      Successfully uninstalled transformers-4.27.2


In [16]:
device = 0 if torch.cuda.is_available() else "cpu"

sentiment_pipe = pipeline("text-classification",
                          model=toxicity_model_name,
                          device=device)
reward_logits_kwargs = {
    "top_k": None, # Return all scores.
    "function_to_apply": "none", # Set to "none" to retrieve raw logits.
    "batch_size": 16
}

reward_probabilities_kwargs = {
    "top_k": None, # Return all scores.
    "function_to_apply": "softmax", # Set to "softmax" to apply softmax and retrieve probabilities.
    "batch_size": 16
}

print("Reward model output:")
print("For non-toxic text")
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(non_toxic_text, **reward_probabilities_kwargs))
print("For toxic text")
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))

Reward model output:
For non-toxic text
[{'label': 'nothate', 'score': 3.114100694656372}, {'label': 'hate', 'score': -2.4896175861358643}]
[{'label': 'nothate', 'score': 0.9963293671607971}, {'label': 'hate', 'score': 0.003670616541057825}]
For toxic text
[{'label': 'hate', 'score': 0.37227070331573486}, {'label': 'nothate', 'score': -0.6921164393424988}]
[{'label': 'hate', 'score': 0.7435280084609985}, {'label': 'nothate', 'score': 0.25647199153900146}]


The outputs are the logits for both nothate (positive) and hate (negative) classes. But PPO will be using logits only of the nothate class as the positive reward signal used to help detoxify the LLM outputs

### Evaluate Toxicity

To evaluate the model before and after fine-tuning/detoxification you need to set up the toxicity evaluation metric. The toxicity score is a decimal value between 0 and 1 where 1 is the highest toxicity.

In [18]:
toxicity_evaluator = evaluate.load("toxicity",
                                    toxicity_model_name,
                                    module_type="measurement",
                                    toxic_label="hate")

In [19]:
toxicity_score = toxicity_evaluator.compute(predictions=[
    non_toxic_text
])

print("Toxicity score for non-toxic text:")
print(toxicity_score["toxicity"])

toxicity_score = toxicity_evaluator.compute(predictions=[
    toxic_text
])

print("\nToxicity score for toxic text:")
print(toxicity_score["toxicity"])

Toxicity score for non-toxic text:
[0.003670616541057825]

Toxicity score for toxic text:
[0.7435289621353149]


This evaluator can be used to compute the toxicity of the dialogues prepared in section 2.1. You will need to pass the test dataset (dataset["test"]), the same tokenizer which was used in that section, the frozen PEFT model prepared in section 2.2, and the toxicity evaluator. It is convenient to wrap the required steps in the function evaluate_toxicity.

In [31]:
def evaluate_toxicity(model,
                      toxicity_evaluator,
                      tokenizer,
                      dataset,
                      num_samples):

    """
    Preprocess the dataset and split it into train and test parts.

    Parameters:
    - model (trl model): Model to be evaluated.
    - toxicity_evaluator (evaluate_modules toxicity metrics): Toxicity evaluator.
    - tokenizer (transformers tokenizer): Tokenizer to be used.
    - dataset (dataset): Input dataset for the evaluation.
    - num_samples (int): Maximum number of samples for the evaluation.

    Returns:
    tuple: A tuple containing two numpy.float64 values:
    - mean (numpy.float64): Mean of the samples toxicity.
    - std (numpy.float64): Standard deviation of the samples toxicity.
    """

    max_new_tokens=100

    toxicities = []
    input_texts = []
    for i, sample in tqdm(enumerate(dataset)):
        input_text = sample["dialogue_text"]

        if i > num_samples:
            break

        input_ids = tokenizer(input_text, return_tensors="pt", padding=True).input_ids
        input_ids = input_ids.to('cuda:0')

        generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                                             tok_k=0.0,
                                             top_p=1.0,
                                             do_sample=True)

        response_token_ids = model.generate(input_ids=input_ids,
                                            generation_config=generation_config)

        generated_text = tokenizer.decode(response_token_ids[0], skip_special_tokens=True)

        toxicity_score = toxicity_evaluator.compute(predictions=[(input_text + " " + generated_text)])

        toxicities.extend(toxicity_score["toxicity"])

    # Compute mean & std using np.
    mean = np.mean(toxicities)
    std = np.std(toxicities)

    return mean, std

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

mean_before_detoxification, std_before_detoxification = evaluate_toxicity(model=ref_model,
                                                                          toxicity_evaluator=toxicity_evaluator,
                                                                          tokenizer=tokenizer,
                                                                          dataset=dataset["test"],
                                                                          num_samples=10)

print(f'toxicity [mean, std] before detox: [{mean_before_detoxification}, {std_before_detoxification}]')


11it [01:36,  8.79s/it]

toxicity [mean, std] before detox: [0.049949133430014954, 0.02331016123718669]


## Perform Fine-Tuning to Detoxify the Summaries

### Initialize PPOTrainer
For the PPOTrainer initialization, you will need a collator. Here it will be a function transforming the dictionaries in a particular way. You can define and test it:

In [22]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}]
print(f'Collator input: {test_data}')
print(f'Collator output: {collator(test_data)}')

Collator input: [{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}]
Collator output: {'key1': ['value1'], 'key2': ['value2'], 'key3': ['value3']}


Set up the configuration parameters. Load the ppo_model and the tokenizer. You will also load a frozen version of the model ref_model. The first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This works as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original LLM.

In [23]:
learning_rate=1.41e-5
max_ppo_epochs=1
mini_batch_size=4
batch_size=16

config = PPOConfig(
    model_name=model_name,
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

ppo_trainer = PPOTrainer(config=config,
                         model=ppo_model,
                         ref_model=ref_model,
                         tokenizer=tokenizer,
                         dataset=dataset["train"],
                         data_collator=collator)

### Fine-Tune the Model
The fine-tuning loop consists of the following main steps:

Get the query responses from the policy LLM (PEFT model).
Get sentiments for query/responses from hate speech RoBERTa model.
Optimize policy with PPO using the (query, response, reward) triplet.
The operation is running if you see the following metrics appearing:

objective/kl: minimize kl divergence,
ppo/returns/mean: maximize mean returns,
ppo/policy/advantages_mean: maximize advantages.

In [24]:
output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

reward_kwargs = {
    "top_k": None, # Return all scores.
    "function_to_apply": "none", # You want the raw logits without softmax.
    "batch_size": 16
}

max_ppo_steps = 10

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    # Break when you reach max_steps.
    if step >= max_ppo_steps:
        break

    prompt_tensors = batch["input_ids"]

    # Get response from FLAN-T5/PEFT LLM.
    summary_tensors = []

    for prompt_tensor in prompt_tensors:
        max_new_tokens = output_length_sampler()

        generation_kwargs["max_new_tokens"] = max_new_tokens
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)

        summary_tensors.append(summary.squeeze()[-max_new_tokens:])

    # This needs to be called "response".
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in summary_tensors]

    # Compute reward outputs.
    query_response_pairs = [q + r for q, r in zip(batch["query"], batch["response"])]
    rewards = sentiment_pipe(query_response_pairs, **reward_kwargs)

    # You use the `nothate` item because this is the score for the positive `nothate` class.
    reward_tensors = [torch.tensor(reward[not_hate_index]["score"]) for reward in rewards]

    # Run PPO step.
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))

0it [00:00, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
1it [00:37, 37.21s/it]

objective/kl: -0.11150603741407394
ppo/returns/mean: 1.1376409530639648
ppo/policy/advantages_mean: 0.011298708617687225
---------------------------------------------------------------------------------------------------


2it [01:15, 37.60s/it]

objective/kl: -0.09044323116540909
ppo/returns/mean: 0.908376932144165
ppo/policy/advantages_mean: 0.022010505199432373
---------------------------------------------------------------------------------------------------


3it [01:38, 31.34s/it]

...

objective/kl: -0.0606873482465744
ppo/returns/mean: 0.9691619277000427
ppo/policy/advantages_mean: 0.04041738063097
---------------------------------------------------------------------------------------------------


9it [04:06, 22.10s/it]

objective/kl: -0.15578478574752808
ppo/returns/mean: 1.076961874961853
ppo/policy/advantages_mean: 0.03799702972173691
---------------------------------------------------------------------------------------------------


10it [04:25, 26.54s/it]

objective/kl: -0.1081928238272667
ppo/returns/mean: 1.0370209217071533
ppo/policy/advantages_mean: 0.015220753848552704
---------------------------------------------------------------------------------------------------


## Evaluate the Model Quantitatively
Load the PPO/PEFT model back in from disk and use the test dataset split to evaluate the toxicity score of the RL-fine-tuned model.

In [32]:
mean_after_detoxification, std_after_detoxification = evaluate_toxicity(model=ppo_model,
                                                                        toxicity_evaluator=toxicity_evaluator,
                                                                        tokenizer=tokenizer,
                                                                        dataset=dataset["test"],
                                                                        num_samples=10)
print(f'toxicity [mean, std] after detox: [{mean_after_detoxification}, {std_after_detoxification}]')

11it [00:22,  2.02s/it]

toxicity [mean, std] after detox: [0.03665001899935305, 0.0310408510653837]


And compare the toxicity scores of the reference model (before detoxification) and fine-tuned model (after detoxification).

In [33]:
mean_improvement = (mean_before_detoxification - mean_after_detoxification) / mean_before_detoxification
std_improvement = (std_before_detoxification - std_after_detoxification) / std_before_detoxification

print(f'Percentage improvement of toxicity score after detoxification:')
print(f'mean: {mean_improvement*100:.2f}%')
print(f'std: {std_improvement*100:.2f}%')

Percentage improvement of toxicity score after detoxification:
mean: 26.63%
std: -33.16%


###  Evaluate the Model Qualitatively

Let's inspect some examples from the test dataset. You can compare the original ref_model to the fine-tuned/detoxified ppo_model using the toxicity evaluator.

In [34]:
batch_size = 20
compare_results = {}

df_batch = dataset["test"][0:batch_size]

compare_results["query"] = df_batch["query"]
prompt_tensors = df_batch["input_ids"]

summary_tensors_ref = []
summary_tensors = []

# Get response from ppo and base model.
for i in tqdm(range(batch_size)):
    gen_len = output_length_sampler()
    generation_kwargs["max_new_tokens"] = gen_len

    summary = ref_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors_ref.append(summary)

    summary = ppo_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors.append(summary)

# Decode responses.
compare_results["response_before"] = [tokenizer.decode(summary_tensors_ref[i]) for i in range(batch_size)]
compare_results["response_after"] = [tokenizer.decode(summary_tensors[i]) for i in range(batch_size)]

# Sentiment analysis of query/response pairs before/after.
texts_before = [d + s for d, s in zip(compare_results["query"], compare_results["response_before"])]
rewards_before = sentiment_pipe(texts_before, **reward_kwargs)
compare_results["reward_before"] = [reward[not_hate_index]["score"] for reward in rewards_before]

texts_after = [d + s for d, s in zip(compare_results["query"], compare_results["response_after"])]
rewards_after = sentiment_pipe(texts_after, **reward_kwargs)
compare_results["reward_after"] = [reward[not_hate_index]["score"] for reward in rewards_after]

100%|██████████| 20/20 [00:48<00:00,  2.41s/it]
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Store and review the results in a DataFrame

In [35]:
pd.set_option('display.max_colwidth', 500)
df_compare_results = pd.DataFrame(compare_results)
df_compare_results["reward_diff"] = df_compare_results['reward_after'] - df_compare_results['reward_before']
df_compare_results_sorted = df_compare_results.sort_values(by=['reward_diff'], ascending=False).reset_index(drop=True)
df_compare_results_sorted

,query,response_before,response_after,reward_before,reward_after,reward_diff
0,"Summarize the following conversation. townperson: No. I don't hear much about it. I avoid a lot of those stories. Just rumors. villager: Look around. Can't you see this magnificent scenery and all the wildlife? These skittering chipmunks and crickets surly have to be magical. townperson: I think it's nature, but I doubt it's enchantment. villager: Nonsense. These tall thick plants and the life that the lake emits....I think there is something special to this place. I knew the villagers were ...",<pad> townperson dislikes the tales of imaginary worlds and even dislikes the villager sayings that his villages aren't nice.</s>,"<pad> Townperson is looking at some negative pictures about their area. They are from Trevno,point north.The villager is from 5 from Trevno,centurialty so near the boundaries.</s>",2.952017,3.612338,0.660321
1,"Summarize the following conversation. hunter: I know where the wolves are sleeping! I can probably get a pup or two! person: Oh no hunter. Are you after me??? hunter: No, I am not after you. I told you I found the wolves person: Oh, I was afraid. I took food from the Lord's harvest. He sent people to get me. hunter: Why are you stealing from the lord's harvest? person: I have no food. He has tons. hunter: Can you not work? Do you not have a trade? person: I did but i was fired. Times are har...",<pad> The person is looking to buy a cub. He can use a chap from the Lord's harvest. He took food from the Lord's harvest. He has no money.</s>,<pad> person hired hunter to find the wolves. Hunter hunter said he found there is still nowhere to eat mice and not the slightest townspeople were certainly hungry. the patient dragged the wolves onto the field and used a...</s>,1.894000,2.513767,0.619767
2,"Summarize the following conversation. villager: Look around. Can't you see this magnificent scenery and all the wildlife? These skittering chipmunks and crickets surly have to be magical. townperson: I think it's nature, but I doubt it's enchantment. villager: Nonsense. These tall thick plants and the life that the lake emits....I think there is something special to this place. I knew the villagers were right all along and I finally can see it now that I am here. townperson: I like the villa...",<pad> villager is from Trevno.</s>,"<pad> The town official tells villager he appreciates the beauty of Lydiante panorama. But the villager quarreles because he doesn't have the right to be here. The villager lives in Trevno, the town north of the forests. Villager decides to live in Katos and doesn't mind being separated from him.</s>",2.978625,3.375443,0.396817
3,"Summarize the following conversation. person: Oh are you not allowed to explore the forest? villager: Oh yes, sometimes someone comes back a bit different...you know, not right in the head. But aside from that, it's a lovely forest. person: Well then that is why you get a bad reputation! Can you not have them see a physician villager: Well, the physicians refuse to see them until an exorcism is performed, and the priests refuse to perform an exorcism because ""none can cleanse the taint of th...",<pad> Villager and others enjoy sex and have secret meetings.</s>,"<pad> villager was once an exorcism of the Earl. A part of human culture, he believes. It's also the practice of exorcism which, it turns out, was misused by an outlaw who had a skin to do sizable amounts of generally good, person's saved may be fishing in it nices ""holy boat"""" that is used to raise, lowly position: the honourarium, annuelled, discounts total already, but Mr. Hill did, at least less painful than the drink, who did ticket sales. Thus once again, three heroic elephantvules hav...",1.397961,1.650126,0.252164
4,"Summarize the following conversation. bird: Oh my many fish here! I wish I could get one customer: Hello bird. I will trade you a fish for the new corn if you can find some for me bird: I can do that

## Conclusion
In our laboratory sessions, we explored the concepts of zero-shot, one-shot, and few-shot inference, observing their impact on the results. Additionally, we delved into some aspects of prompt engineering. Utilizing various Google models as foundational structures, we enhanced our understanding of in-context learning. Moreover, we employed an instruction fine-tuned model, commonly referred to as the instruct model, and conducted comparisons of the outcomes both manually and through quantitative measures, specifically using ROUGE scores. Our experiments also included the implementation of PEFT models, which revealed variations in the summarization results. Furthermore, we integrated a reward model aimed at diminishing the toxicity in the generated text

## References
https://www.coursera.org/learn/generative-ai-with-llms <br>
https://huggingface.co/google/flan-t5-small <br>
https://huggingface.co/google/flan-t5-base <br>